In [14]:
import numpy as np
import do_mpc

model_type = 'continuous'
model = do_mpc.model.Model(model_type)

# Define the system's state variables
theta = model.set_variable('_x', 'theta')  # Servo's angular position
omega = model.set_variable('_x', 'omega')  # Servo's angular velocity

# Define the control input
control_omega = model.set_variable('_u', 'control_omega')  # Angular velocity command

# Time varying parameters
target_theta = model.set_variable('_tvp', 'target_theta')
target_omega = model.set_variable('_tvp', 'target_omega')

# System equations
model.set_rhs('theta', omega)
model.set_rhs('omega', control_omega)

model.setup()


In [15]:
mpc = do_mpc.controller.MPC(model)

setup_mpc = {
    'n_horizon': 20,
    't_step': 0.005,
    'state_discretization': 'collocation',
    'collocation_type': 'radau',
    'collocation_deg': 2,
    'collocation_ni': 2,
    'store_full_solution': True,
}

mpc.set_param(**setup_mpc)

# Define the objective function
w_theta = 1.0  # Weight for angular position error
w_omega = 0.1  # Weight for angular velocity error

# Objective function
lterm = w_theta * (model.x['theta'] - model.tvp['target_theta'])**2
lterm += w_omega * (model.x['omega'] - model.tvp['target_omega'])**2
mpc.set_objective(lterm=lterm, mterm=lterm)

# Bounds for the control input
yaw_omega_max = 100 * 360 / 60  # 100 RPM to deg/s
mpc.bounds['lower','_u', 'control_omega'] = -yaw_omega_max
mpc.bounds['upper','_u', 'control_omega'] = yaw_omega_max

tvp_template = mpc.get_tvp_template()

def tvp_fun(t_now):
    amplitude = 45
    frequency = 0.5

    tvp_template['_tvp', :, 'target_theta'] = amplitude * np.sin(2 * np.pi * frequency * t_now)
    tvp_template['_tvp', :, 'target_omega'] = amplitude * 2 * np.pi * frequency * np.cos(2 * np.pi * frequency * t_now)
    
    return tvp_template

mpc.set_tvp_fun(tvp_fun)

mpc.setup()


/usr/local/lib/python3.11/site-packages/do_mpc/controller/_mpc.py:885: UserWarning: rterm was not set and defaults to zero. Changes in the control inputs are not penalized. Can lead to oscillatory behavior.
  warnings.warn('rterm was not set and defaults to zero. Changes in the control inputs are not penalized. Can lead to oscillatory behavior.')


In [16]:
simulator = do_mpc.simulator.Simulator(model)
simulator.set_param(t_step=0.005)

# Define initial state for the simulator
initial_state = np.array([[0], [0]])  # Replace with your initial state
simulator.x0 = initial_state

simulator.setup()


Exception: You have not supplied a function to obtain the time-varying parameters defined in model. Use .set_tvp_fun() prior to setup.

In [ ]:
for k in range(100):  # Simulate for 100 steps
    u0 = mpc.make_step(simulator.x0)
    y_next = simulator.make_step(u0)